In [ ]:
dt_02 = True
i_model = 4
from scipy.stats import chi2
import numpy as np
import matplotlib.pyplot as plt

qunatiles_ground_data = np.load(f"results/reference_values_ranges{"_02" if dt_02 else ""}.npy")
qunatiles_ground_data = qunatiles_ground_data[:, i_model] #select model
qunatiles_ground_data[:, 0] = 1 - qunatiles_ground_data[:, 0] # transform attractor inclusion to attractor exclusion
quantiles = np.quantile(qunatiles_ground_data, 0.95 , axis=0)
quantiles[-1] = chi2.isf(0.05, df=10) # there is a theoretical value for the threshold in the case of Heikki

In [ ]:
quantiles

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

data = np.load("results/create_figure_y.npz")
measures = data["measures"]
predictions = data["predictions_closed_loop"]
reference = data["data_closed_loop"]

In [ ]:
measures.shape, predictions.shape, reference.shape

In [ ]:
np.sum((measures[:, 5:] < quantiles[1:]).all(axis=-1))

In [ ]:
indicies = np.where((measures[:, 5:] < quantiles[1:]).all(axis=-1))[0]
measures, predictions, reference = measures[indicies], predictions[indicies], reference[indicies]

In [ ]:
from mpl_toolkits.mplot3d import proj3d

ax = plt.subplot(111, projection="3d")
proj = lambda x, ax=ax: proj3d.proj_transform(*x.T, ax.get_proj())[:2]

from matplotlib.colors import LinearSegmentedColormap
import matplotlib.colors as mcolors

################## colormap ###################

cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 1.0, 1.0),
                   (1.0, 1.0, 1.0)),

         'green': ((0.0, 116./255., 116./255.),
                   (0.5, 1., 1.),
                   (1.0, 121./255., 121./255.)),

         'blue':  ((0.0, 122./255., 122./255.),
                   (0.5, 1., 1.0),
                   (1.0, 0.0, 0.0))
        }
       

TUI = LinearSegmentedColormap('TUI', cdict)
norm = mcolors.Normalize(vmin=0, vmax=1)  

In [ ]:
cm = 1/2.54
n = 6
fig, axs = plt.subplots(3, n, figsize=(21*cm-5*cm, 5*cm), dpi=300, height_ratios=[1, 1, 0.5])

for ax_row, index in zip(axs.T, range(n)):
    (prediction, data, measure) = predictions[index], reference[index], measures[index]

    for ax in ax_row:
        ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
        ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

        # Hide the spines (borders)
        for spine in ['top', 'bottom', 'left', 'right']:
            ax.spines[spine].set_visible(False)
    
    ax_row[0].plot(*proj(data), lw=0.1, rasterized=True, color=TUI(norm(1)), alpha=0.8)
    ax_row[0].plot(*proj(prediction), lw=0.1, rasterized=True, color=TUI(0), alpha=0.8)
    ax_row[1].hist2d(*proj(prediction), rasterized=True, bins=64, cmap="Grays")
    ax_row[1].set_xlim(ax_row[0].get_xlim())
    ax_row[1].set_ylim(ax_row[0].get_ylim())

    ax_row[2].axis([0, 1, 0, 4])
    for i, (value, name) in enumerate(zip(measure[4:], ("AExc", "ADev", "TVar", "GCI"))):
        if name == "ADev" or name == "GCI":
            text = f"{value:.0f}"
        elif name == "TVar":
            text = f"{value:.3f}"
        else:
            text = f"{value:.2f}"
        ax_row[2].text(0.5, i, text, horizontalalignment="left", fontsize=4, color='black')
        ax_row[2].text(0.2, i, f"{name}: ", horizontalalignment="left", fontsize=4, color='black')
    # ax = axs[1]
    # ax.hist2d(*proj(prediction), rasterized=True, bins=64, cmap="Grays")
    # plt.show()
plt.tight_layout()
plt.savefig("pictures/illustrations/good_examples.pdf")

In [ ]:
from matplotlib import colors


cm = 1/2.54
n = 6
fig, axs = plt.subplots(3, n, figsize=(21*cm-5*cm, 4.5*cm), dpi=300, height_ratios=[1, 1, 0.5])
global norm_colorbar

for ax_row, index in zip(axs.T, range(n)):
    (prediction, data, measure) = predictions[index], reference[index], measures[index]

    if index == 0:
        # color pallet
        cmap = plt.cm.Greys.copy()
        cmap.set_over('#fc03f8')     # For values > vmax
        # find common colorbar
        max_counts, x_edges, y_edges = np.histogram2d(*proj(prediction), bins=64)
        norm_colorbar = colors.Normalize(0, max_counts.max()*1.2, clip=False)
        _, _, _, hist = ax.hist2d(*proj(prediction), rasterized=True, bins=[x_edges, y_edges], norm=norm_colorbar, cmap=cmap)
        #cbar = miniplot_grid.cbar_axes[0].colorbar(hist, extend="max", cmap=cmap)
        #cbar.set_label("Counts", size=12)

    for ax in ax_row:
        ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
        ax.tick_params(axis='y', which='both', left=False, right=False, labelleft=False)

        # Hide the spines (borders)
        for spine in ['top', 'bottom', 'left', 'right']:
            ax.spines[spine].set_visible(False)
    
    ax_row[0].plot(*proj(data), lw=0.1, rasterized=True, color=TUI(norm(1)), alpha=0.8)
    ax_row[0].plot(*proj(prediction), lw=0.1, rasterized=True, color=TUI(0), alpha=0.8)
    ax_row[1].hist2d(*proj(prediction), rasterized=True, bins=64, cmap=cmap, norm=norm_colorbar)
    ax_row[1].set_xlim(ax_row[0].get_xlim())
    ax_row[1].set_ylim(ax_row[0].get_ylim())

    ax_row[2].axis([0, 1, 0, 4])
    for i, (value, name) in enumerate(zip(measure[4:], ("AExc", "ADev", "TVar", "GCI"))):
        if name == "ADev" or name == "GCI":
            text = f"{value:.0f}"
        elif name == "TVar":
            text = f"{value:.3f}"
        else:
            text = f"{value:.2f}"
        ax_row[2].text(0.5, 1.3*i, text, horizontalalignment="left", fontsize=5, color='black')
        ax_row[2].text(0.2, 1.3*i, f"{name}: ", horizontalalignment="left", fontsize=5, color='black')
    # ax = axs[1]
    # ax.hist2d(*proj(prediction), rasterized=True, bins=64, cmap="Grays")
    # plt.show()
plt.tight_layout()
cbar = fig.colorbar(hist, ax=axs.ravel().tolist(), extend="max", cmap=cmap)
cbar.set_label("Counts", size=5)
cbar.set_ticklabels(cbar.get_ticks(), size=5)
cbar.outline.set_linewidth(0.3)
plt.savefig("pictures/illustrations/good_examples.pdf", bbox_inches='tight')

In [ ]:
cbar.vmin, cbar.vmax